# Geometrias das faixas de pedestre

In [154]:
## TODO

# Gerar pontos para todas as faixas
# Relacionar espacialmente os pontos com os polígonos de vias
# Agrupar por proximidade usando o DBSCAN
# DEsenhar um retangulo orientado em cada cluster
# Encontrar a linha de percurso da faixa de pedestre
# Estender o polígono até as calçadas
# Salvar os polígonos

In [44]:
import geopandas as gpd
import rasterio
import numpy as np
import os
from rasterio.enums import Resampling

In [45]:
dataset = rasterio.open('resultados/raster/faixas-3314-452.jp2')

In [101]:
data = dataset.read()

In [106]:
[len(x) for x in np.where(data == 1)]

[55455, 55455, 55455]

In [133]:
faixas_ij = np.array([np.where(data == 1)[2], np.where(data == 1)[1]]).transpose()

In [136]:
faixas_ij

array([[3768,   24],
       [3769,   24],
       [3770,   24],
       ...,
       [ 296, 4883],
       [ 297, 4883],
       [ 298, 4883]])

In [137]:
t = dataset.transform
t

Affine(0.12000000000000001, 0.0, 332566.5289866985,
       0.0, -0.12000000000000001, 7392105.466007747)

In [146]:
faixas_xy = faixas_ij * (t[0], t[4]) + (t[2], t[5]) + (t[0]/2, t[4]/2)
faixas_xy

array([[ 333018.7489867 , 7392102.52600775],
       [ 333018.8689867 , 7392102.52600775],
       [ 333018.9889867 , 7392102.52600775],
       ...,
       [ 332602.1089867 , 7391519.44600775],
       [ 332602.2289867 , 7391519.44600775],
       [ 332602.3489867 , 7391519.44600775]])

In [150]:
geometries = gpd.points_from_xy(x=faixas_xy[:, 0], y=faixas_xy[:, 1], crs='EPSG:31983')

In [153]:
gpd.GeoDataFrame(geometry=geometries).to_file('resultados/vetor/faixas-3314-452.gpkg', driver='GPKG')